In [ ]:
!pip install pandas
!pip install matplotlib
!pip install seaborn

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv('/content/OnlineShoppersС.csv')

In [ ]:
data

In [ ]:
data.info()

In [ ]:
data = data.drop(columns=['Unnamed: 18'])

In [ ]:
data.duplicated().sum()

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
data.duplicated().sum()

In [ ]:
columns = ['Month','VisitorType']
for column in columns:
  data[column] = data[column].factorize()[0]

In [ ]:
data

In [ ]:
data.info()

In [ ]:
data.describe()

визуализация

гистограммы

In [ ]:
data.hist(bins=30, figsize=(12, 8))
plt.tight_layout()
plt.show()

ящики с усами

In [ ]:
data.columns

In [ ]:
for column in ['Administrative', 'Administrative_Duration', 'Informational',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay', 'Month',
       'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType']:
  plt.figure(figsize=(8, 6))
  plt.boxplot(data[column])
  plt.title(column)
  plt.show()

In [ ]:
def replace_outliers_with_mean(df):
    for column in ['Administrative_Duration', 'Informational',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'PageValues']:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        mean_value = df[column].mean()

        df[column] = df[column].apply(lambda x: mean_value if x < lower_bound or x > upper_bound else x)

    return df

data = replace_outliers_with_mean(data)

data.head()

In [ ]:
for column in ['Administrative', 'Administrative_Duration', 'Informational',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay', 'Month',
       'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType']:
  plt.figure(figsize=(8, 6))
  plt.boxplot(data[column])
  plt.title(column)
  plt.show()

диаграммы рассеяния

In [ ]:
['Administrative', 'Administrative_Duration', 'Informational',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay', 'Month',
       'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType']
pairs = [
    ('Administrative', 'BounceRates'),
    ('Administrative', 'ExitRates'),
    ('Administrative', 'PageValues'),
    ('Administrative', 'SpecialDay'),
    ('Administrative', 'Month'),
]

for x_col, y_col in pairs:
    plt.figure(figsize=(6, 4))
    plt.scatter(data[x_col], data[y_col])
    plt.xlabel(x_col)
    plt.ylabel(y_col)
    plt.title(f'{y_col} vs {x_col}')
    plt.tight_layout()
    plt.show()

In [ ]:
corr_matrix = data.corr()

plt.figure(figsize=(12, 12))

sns.heatmap(corr_matrix, annot=True, linewidths=0.5)

plt.tight_layout()
plt.show()

модели


In [ ]:
X = data.drop(columns=['Revenue'])
y = data['Revenue']

снижение размерности

In [ ]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold=0.1)

X_reduced = selector.fit_transform(X)

selected_columns = X.columns[selector.get_support()]

print(f"Оставшиеся признаки ({len(selected_columns)}): {list(selected_columns)}")

X = data[selected_columns]
X

балансировка

In [ ]:
pd.Series(y).value_counts()

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd

smote = SMOTE(random_state=42)
X_sm, y_sm = smote.fit_resample(X, y)

sampling_strategy = {0: 5000, 1: 5000}

undersample = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)
X, y = undersample.fit_resample(X_sm, y_sm)

print(pd.Series(y).value_counts())
print(f"Итоговый размер: {len(X)}")


из моей ЛР3

In [ ]:
from imblearn.under_sampling import TomekLinks, RandomUnderSampler
import pandas as pd

tomek = TomekLinks()
X_cleaned, Y_cleaned = tomek.fit_resample(X, Y)

sampling_strategy = {0: 1000, 1: 1000}

undersample = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)
X, Y = undersample.fit_resample(X_cleaned, Y_cleaned)

print(pd.Series(Y).value_counts())
print(f"Итоговый размер: {len(Y)}")


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

логистическая регрессия

In [ ]:
from scipy.stats import uniform
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV

param_distributions = {
    'C':        [0.001, 0.01, 0.1, 1, 10],
    'l1_ratio': [0.0, 0.25, 0.5, 0.75, 1.0]
}

rs = RandomizedSearchCV(
    estimator=LogisticRegression(
        penalty='elasticnet',
        solver='saga',
        max_iter=1000,
        random_state=42
    ),
    param_distributions=param_distributions,
    n_iter=10,
    cv=5,
    scoring='accuracy',
    random_state=42,
    n_jobs=-1
)

rs.fit(X_train, y_train)
print("Best params:", rs.best_params_)
print("Test accuracy:", rs.score(X_test, y_test))


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, classification_report
)

# Предполагается, что X_train, X_test, y_train, y_test уже определены

# 1) Инициализация и обучение модели
model = LogisticRegression(
    penalty='elasticnet',
    solver='saga',
    C=0.001,
    l1_ratio=0.0,
    max_iter=1000,
    random_state=42
).fit(X_train, y_train)

# 2) Предсказания меток и вероятностей
y_pred_train = model.predict(X_train)
y_pred_test  = model.predict(X_test)
y_proba_train = model.predict_proba(X_train)[:, 1]
y_proba_test  = model.predict_proba(X_test)[:, 1]

# 3) Метрики на train
print("=== Train metrics ===")
print("Accuracy : ", accuracy_score(y_train, y_pred_train))
print("Precision: ", precision_score(y_train, y_pred_train))
print("Recall   : ", recall_score(y_train, y_pred_train))
print("F1-score : ", f1_score(y_train, y_pred_train))
print("ROC AUC  : ", roc_auc_score(y_train, y_proba_train))
print("\nClassification Report (Train):")
print(classification_report(y_train, y_pred_train))

# 4) Метрики на test
print("=== Test metrics ===")
print("Accuracy : ", accuracy_score(y_test, y_pred_test))
print("Precision: ", precision_score(y_test, y_pred_test))
print("Recall   : ", recall_score(y_test, y_pred_test))
print("F1-score : ", f1_score(y_test, y_pred_test))
print("ROC AUC  : ", roc_auc_score(y_test, y_proba_test))
print("\nClassification Report (Test):")
print(classification_report(y_test, y_pred_test))


дерево решений

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV, train_test_split

param_distributions = {
    'max_depth':        randint(3, 15),
    'min_samples_split':randint(2, 20),
    'min_samples_leaf': randint(1, 10)
}

rs = RandomizedSearchCV(
    estimator=DecisionTreeClassifier(random_state=42),
    param_distributions=param_distributions,
    n_iter=20,
    cv=3,
    scoring='accuracy',
    random_state=42,
    n_jobs=-1
)
rs.fit(X_train, y_train)

print("Best params:", rs.best_params_)
print("Test accuracy:", rs.score(X_test, y_test))


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, classification_report
)

# Предполагается, что X_train, X_test, y_train, y_test уже определены

# Инициализация модели с найденными гиперпараметрами
model = DecisionTreeClassifier(
    max_depth=6,
    min_samples_split=13,
    min_samples_leaf=7,
    random_state=42
)

# Обучение
model.fit(X_train, y_train)

# Предсказания и вероятности
y_pred_train = model.predict(X_train)
y_pred_test  = model.predict(X_test)
y_proba_train = model.predict_proba(X_train)[:, 1]
y_proba_test  = model.predict_proba(X_test)[:, 1]

# Метрики на train
print("=== Train metrics ===")
print("Accuracy : ", accuracy_score(y_train, y_pred_train))
print("Precision: ", precision_score(y_train, y_pred_train))
print("Recall   : ", recall_score(y_train, y_pred_train))
print("F1-score : ", f1_score(y_train, y_pred_train))
print("ROC AUC  : ", roc_auc_score(y_train, y_proba_train))
print("\nClassification Report (Train):")
print(classification_report(y_train, y_pred_train))

# Метрики на test
print("=== Test metrics ===")
print("Accuracy : ", accuracy_score(y_test, y_pred_test))
print("Precision: ", precision_score(y_test, y_pred_test))
print("Recall   : ", recall_score(y_test, y_pred_test))
print("F1-score : ", f1_score(y_test, y_pred_test))
print("ROC AUC  : ", roc_auc_score(y_test, y_proba_test))
print("\nClassification Report (Test):")
print(classification_report(y_test, y_pred_test))


Light GBM

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from lightgbm import LGBMClassifier
from scipy.stats import randint, uniform


param_distributions = {
    'num_leaves':        randint(20, 150),
    'max_depth':         randint(3, 12),
    'learning_rate':     uniform(loc=0.01, scale=0.29)
}

rs = RandomizedSearchCV(
    estimator=LGBMClassifier(random_state=42, n_jobs=-1),
    param_distributions=param_distributions,
    n_iter=20,
    cv=3,
    scoring='accuracy',
    random_state=42,
    n_jobs=-1
)

rs.fit(X_train, y_train)

print("Best params:", rs.best_params_)
print("Test accuracy:", rs.score(X_test, y_test))


In [ ]:
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

model = LGBMClassifier(
    learning_rate=0.29127385712697834,
    max_depth=8,
    num_leaves=149,
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)
model.fit(X_train, y_train)

y_pred_train = model.predict(X_train)
y_proba_train = model.predict_proba(X_train)[:, 1]
y_pred_test  = model.predict(X_test)
y_proba_test = model.predict_proba(X_test)[:, 1]

acc_train     = accuracy_score(y_train, y_pred_train)
prec_train    = precision_score(y_train, y_pred_train)
rec_train     = recall_score(y_train, y_pred_train)
f1_train      = f1_score(y_train, y_pred_train)
roc_auc_train = roc_auc_score(y_train, y_proba_train)

acc_test     = accuracy_score(y_test, y_pred_test)
prec_test    = precision_score(y_test, y_pred_test)
rec_test     = recall_score(y_test, y_pred_test)
f1_test      = f1_score(y_test, y_pred_test)
roc_auc_test = roc_auc_score(y_test, y_proba_test)

print("=== Train Metrics ===")
print(f"Accuracy : {acc_train:.4f}")
print(f"Precision: {prec_train:.4f}")
print(f"Recall   : {rec_train:.4f}")
print(f"F1-score : {f1_train:.4f}")
print(f"ROC AUC  : {roc_auc_train:.4f}\n")

print("=== Test Metrics ===")
print(f"Accuracy : {acc_test:.4f}")
print(f"Precision: {prec_test:.4f}")
print(f"Recall   : {rec_test:.4f}")
print(f"F1-score : {f1_test:.4f}")
print(f"ROC AUC  : {roc_auc_test:.4f}")


Accuracy
– Когда классы примерно сбалансированы и ложноположительные (FP) и ложноотрицательные (FN) ошибки едва ли имеют разную «стоимость».

Precision
– Если дорого выдавать «ложные тревоги» (FP). Например, спам-фильтр: лучше не пометить важное письмо как спам.

Recall
– Когда критично не пропустить «положительный» класс (FN). Например, медицинская диагностика или обнаружение мошенничества.

F1-score
– Если нужно единое число-компромисс между Precision и Recall и нет явного приоритета ни на FP, ни на FN.

ROC-AUC
– Для общего сравнения моделей по качеству ранжирования (вероятностных предсказаний), особенно при умеренном балансе классов.

PR-AUC
– Когда положительный класс редок (сильный дисбаланс) и вы хотите оценить модель в области низких долей «1».

деплой

In [ ]:
import pandas as pd

# Пример одного нового наблюдения
new_data = pd.DataFrame([{
    'Administrative': 5,
    'Administrative_Duration': 120.5,
    'Informational_Duration': 10.0,
    'ProductRelated': 20,
    'ProductRelated_Duration': 800.0,
    'PageValues': 1.5,
    'Month': 3,
    'OperatingSystems': 2,
    'Browser': 3,
    'Region': 1,
    'TrafficType': 4,
    'VisitorType': 1,
    'Weekend': False
}])

# Предсказание класса и вероятность для этого единственного объекта
pred = model.predict(new_data)[0]
proba = model.predict_proba(new_data)[0, 1]

print(f"Предсказанный класс: {int(pred)}")
print(f"Вероятность класса 1: {proba:.3f}")